<a href="https://colab.research.google.com/github/marcospiau/ia368-dd-dl4ir/blob/main/aula05-doc2query/aula05_doc2query_indexing_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook will be used to index texts and eval results

# TL;DR: GO to `Calculating metrics for runs` for results

# Installs and imports

In [1]:
!free -mh
!lscpu

              total        used        free      shared  buff/cache   available
Mem:           35Gi       964Mi        25Gi       1.0Mi       8.5Gi        33Gi
Swap:            0B          0B          0B
Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          40
On-line CPU(s) list:             0-39
Thread(s) per core:              2
Core(s) per socket:              20
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           63
Model name:                      Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:                        0
CPU MHz:                         2299.998
BogoMIPS:                        4599.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d c

In [2]:
%%capture
!wget -nc https://raw.githubusercontent.com/marcospiau/ia368-dd-dl4ir/main/scripts/install_anserini.sh && chmod +x install_anserini.sh && time ./install_anserini.sh

In [3]:
%%capture
!pip install -q ftfy polars toolz cytoolz transformers datasets gcsfs
!pip install -U t5[gcp,cache-tasks]==0.9.3
!pip install -U jaxlib
!sudo apt install -qq tree htop

**PS.: notebook must be restarted after packages above are installed.**

# gcloud authentication

In [1]:
import os
from google.colab import auth

os.environ['USE_AUTH_EPHEM'] = '0'
auth.authenticate_user(clear_output=False)

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=DgAUGjJhkg5Jb5qcPJU2HG3KDuvSS5&prompt=consent&access_type=offline&code_challenge=0cHd61StG_TiZJMEN8Nz-hEtA9s0phsG-E1pRVC-zZo&code_challenge_method=S256

Enter authorization code: 4/0AVHEtk7eD4PHy1PajvHuflFR5m6oT8zxWJBpr962z-kZrV_wzQ7Fc72_vh8WlPmRtPTcYg


In [2]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=fV8ByYSipdHheYmIarvD7CItVPKKcA&prompt=consent&access_type=offline&code_challenge=fUD6NFUIvqOSAOZjmeQQSH6b8QLoi0GzKl4wx0G6WIo&code_challenge_method=S256

Enter authorization code: 4/0AVHEtk6B-qQ6SayvEK0CtYQEdc3GUIh6gHIjIpGEleqybEqa8ZlRtyowfmso1aOQPCffZQ

You are now logged in as [m136825@dac.unicamp.br].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


# Preparing qrels and queries files

In [3]:
import os
import datasets
import toolz
import multiprocessing as mp
import pandas as pd
from collections import Counter, defaultdict
import itertools
import polars as pl

import more_itertools
from etils import epath
from pathlib import Path
import functools
from tqdm import tqdm
import multiprocessing as mp


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

os.environ['POLARS_FMT_STR_LEN']='1000'

## Sequence truncation analysis - SCRATCH

In [ ]:
from transformers import T5Tokenizer

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
def concat_title_and_text(ex):
    return {'title_and_text': f"{ex['title']}. {ex['text']}"}

def encode(batch):
    lengths = tokenizer(batch['title_and_text'], return_length=True)['length']
    return {'lengths': lengths}

In [ ]:
ds_corpus = datasets.load_dataset('BeIR/trec-covid', 'corpus')['corpus']
ds_corpus = ds_corpus.map(concat_title_and_text)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
token_lengths = pd.Series(ds_corpus.map(encode, batched=True, num_proc=mp.cpu_count())['lengths'])

In [ ]:
tokenizer(ds_corpus.select(range(1))['title_and_text'])

{'input_ids': [[14067, 753, 13, 1543, 18, 1409, 1926, 499, 509, 21178, 9, 30195, 15, 13315, 44, 2671, 28508, 9, 702, 172, 636, 4457, 6, 1022, 26, 26, 9, 107, 6, 11279, 13849, 5, 3, 10539, 683, 14196, 8087, 10, 100, 29825, 5059, 1132, 8788, 8, 29969, 1863, 11, 3739, 753, 13, 1283, 1221, 28, 1543, 18, 1409, 1926, 499, 509, 21178, 9, 30195, 15, 13315, 44, 2671, 28508, 9, 702, 172, 636, 4457, 6, 1022, 26, 26, 9, 107, 6, 11279, 13849, 5, 3, 24506, 6299, 3592, 10, 18027, 28, 1465, 283, 5, 30195, 15, 9757, 45, 19944, 19622, 7, 45, 1762, 6622, 190, 1882, 6260, 130, 4313, 190, 8, 5893, 6420, 6427, 3187, 5, 15054, 7, 13, 1221, 130, 9112, 5, 3, 12200, 4254, 4578, 10, 1283, 1221, 130, 4313, 6, 5400, 13642, 15967, 6210, 13, 4068, 831, 7209, 5, 1377, 13315, 14156, 15967, 6210, 130, 573, 18, 9, 75, 1169, 1271, 5, 37, 7952, 4161, 66, 1246, 1637, 68, 47, 167, 1017, 16, 9806, 7, 6918, 15967, 6210, 11, 554, 18, 6646, 502, 4743, 15967, 6210, 5, 94, 6935, 215, 18, 7775, 68, 47, 167, 1017, 16, 8, 1590, 41, 

In [ ]:
token_lengths.describe().to_frame().round(2)

,0
count,171332.00
mean,266.93
std,254.57
min,3.00
25%,57.00
50%,268.00
75%,406.00
max,42655.00


In [ ]:
token_lengths.gt(512).value_counts().to_frame()

,0
False,154953
True,16379


In [ ]:
ds_qrels = datasets.load_dataset('BeIR/trec-covid-qrels')['test']
ds_qrels

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/BeIR___csv/BeIR--trec-covid-qrels-1766e3af5b0b856a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['query-id', 'corpus-id', 'score'],
    num_rows: 66336
})

In [ ]:
a = ds_qrels.to_pandas().apply(lambda x: (str(x['query-id']), str(x['corpus-id'])), axis=1)
a = sorted(a)
Counter(a).most_common(10)

[(('1', '005b2j4b'), 1),
 (('1', '00fmeepz'), 1),
 (('1', '0194oljo'), 1),
 (('1', '021q9884'), 1),
 (('1', '02f0opkr'), 1),
 (('1', '02say5f1'), 1),
 (('1', '0376d6vf'), 1),
 (('1', '047xpt2c'), 1),
 (('1', '04ftw7k9'), 1),
 (('1', '05vx82oo'), 1)]

In [ ]:
b = pd.read_csv('tools/topics-and-qrels/qrels.beir-v1.0.0-trec-covid.test.txt', sep=' ', header=None).apply(lambda x: (str(x[0]), str(x[2])), axis=1)
b = sorted(b)


In [ ]:
# There are two ids missing from qrels in anserini tools
print(set(a) - set(b))
print(set(b) - set(a))

{('50', 'svo94kuo'), ('38', '9hbib8b3')}
set()


## Convert qrels to TREC format

In [4]:
def convert_qrels_to_trec_format(output_file):
    def to_line(ex):
        # qid 0 docid score
        return '{} 0 {} {}\n'.format(ex['query-id'], ex['corpus-id'],
                                   ex['score'])
    ds = datasets.load_dataset('BeIR/trec-covid-qrels')['test']
    with open(output_file, 'w') as f:
        for item in ds.to_list():
            line = to_line(item)
            f.write(line)

QRELS_FILE = 'trec-covid-qrels_trec_format.txt'

convert_qrels_to_trec_format(QRELS_FILE)
!wc -l {QRELS_FILE}
!head {QRELS_FILE}

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/BeIR___csv/BeIR--trec-covid-qrels-1766e3af5b0b856a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

66336 trec-covid-qrels_trec_format.txt
1 0 005b2j4b 2
1 0 00fmeepz 1
1 0 g7dhmyyo 2
1 0 0194oljo 1
1 0 021q9884 1
1 0 02f0opkr 1
1 0 047xpt2c 0
1 0 04ftw7k9 0
1 0 pl9ht0d0 0
1 0 05vx82oo 0


## Convert queries to TREC format

~Queries are already in `anserini/src/main/resources/topics-and-qrels/topics.beir-v1.0.0-trec-covid.test.tsv.gz`, so we are going to use it instead of downloading from HuggingFace.~



In [5]:
def convert_queries_to_trec_format(output_file):
    ds = datasets.load_dataset('BeIR/trec-covid', 'queries')['queries']
    ds.to_csv(output_file, header=None, sep='\t', columns=['_id', 'text'])

QUERIES_FILE = 'trec-covid-queries_trec_format.txt'

convert_queries_to_trec_format(QUERIES_FILE)
!wc -l {QUERIES_FILE}
!cat {QUERIES_FILE}

Generating queries split: 0 examples [00:00, ? examples/s]

Dataset trec-covid downloaded and prepared to /root/.cache/huggingface/datasets/BeIR___trec-covid/queries/0.0.0/093f1fe2ffa7a9c72fa48239c8f279b51d6b171abd77737c7fd1406125307599. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

50 trec-covid-queries_trec_format.txt
1	what is the origin of COVID-19
2	how does the coronavirus respond to changes in the weather
3	will SARS-CoV2 infected people develop immunity? Is cross protection possible?
4	what causes death from Covid-19?
5	what drugs have been active against SARS-CoV or SARS-CoV-2 in animal studies?
6	what types of rapid testing for Covid-19 have been developed?
7	are there serological tests that detect antibodies to coronavirus?
8	how has lack of testing availability led to underreporting of true incidence of Covid-19?
9	how has COVID-19 affected Canada
10	has social distancing had an impact on slowing the spread of COVID-19?
11	what are the guidelines for triaging patients infected with coronavirus?
12	what are best practices in hospitals and at home in maintaining quarantine?
13	what are the transmission routes of coronavirus?
14	what evidence is there related to COVID-19 super spreaders
15	how long can the coronavirus live outside the body
16	how long doe

# Download pre-built index from pyserini

Following instructions, we will download the pre-built index `beir-v1.0.0-trec-covid-flat`.

In [6]:
from pyserini.index.lucene import IndexReader
from pyserini.search.lucene import LuceneSearcher
from pyserini.util import download_prebuilt_index

Just download the file:

In [7]:
PREBUILT_INDEX_DIR = download_prebuilt_index('beir-v1.0.0-trec-covid.flat',
                                             verbose=True)
print(f'PREBUILT_INDEX_DIR is {PREBUILT_INDEX_DIR}')
!ls -lht {PREBUILT_INDEX_DIR}

lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz: 216MB [00:02, 95.3MB/s]                          


Extracting /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz into /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade...
PREBUILT_INDEX_DIR is /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade
total 258M
-rw-rw-r-- 1 2537 2603  340 Nov 16 15:21 _0.fnm
-rw-rw-r-- 1 2537 2603  19M Nov 16 15:21 _0_Lucene90_0.doc
-rw-rw-r-- 1 2537 2603  22M Nov 16 15:21 _0_Lucene90_0.pos
-rw-rw-r-- 1 2537 2603 3.6M Nov 16 15:21 _0_Lucene90_0.tim
-rw-rw-r-- 1 2537 2603  81K Nov 16 15:21 _0_Lucene90_0.tip
-rw-rw-r-- 1 2537 2603  306 Nov 16 15:21 _0_Lucene90_0.tmd
-rw-rw-r-- 1 2537 2603  516 Nov 16 15:21 _0.si
-rw-rw-r-- 1 2537 2603  154 Nov 16 15:21 segments_1
-rw-rw-r-- 1 2537 2603  242 Nov 16 15:21 _0.fdm
-rw-rw-r-- 1 2537 2603 127M Nov 16 15:21 _0.fdt
-rw-rw-r-- 1 2537 2603  12K Nov 16 15:21 _0.fdx
-rw-rw-r-- 1 2537 2603 1.4M 

Load index and show statistics:

In [8]:
index_reader = IndexReader.from_prebuilt_index('beir-v1.0.0-trec-covid.flat',
                                               verbose=True)
index_reader

Attempting to initialize pre-built index beir-v1.0.0-trec-covid.flat.
/root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade already exists, skipping download.
Initializing beir-v1.0.0-trec-covid.flat...
{'total_terms': 20822821, 'documents': 171331, 'non_empty_documents': 171331, 'unique_terms': 202648}
Index passes consistency checks against pre-built index 'beir-v1.0.0-trec-covid.flat'!


# Validating results from anserini regressions on Beir TREC Covid

We will first attempt to replicate Anserini's results for TREC-COVID. Essentially, we will follow the instructions provided in [Anserini Regressions: BEIR (v1.0.0) — TREC-COVID](https://github.com/castorini/anserini/blob/master/docs/regressions-beir-v1.0.0-trec-covid-flat.md).

Another important file is this YAML, which contains information about evaluation and index building: https://github.com/castorini/anserini/blob/master/src/main/resources/regression/beir-v1.0.0-trec-covid-flat.yaml

In [9]:
import shlex
import subprocess

TREC_EVAL_BIN_PATH = './tools/eval/trec_eval.9.0.4/trec_eval'
# Essa função foi escrita usando o github copilot
def get_trec_eval_metrics(flags, qrels_path, results_path):
    """Runs trec_eval and returns the results as a dictionary.

    Args:
        flags (str): Flags to pass to trec_eval.
        qrels_path (str): Path to the qrels file.
        results_path (str): Path to the results file.

    Returns:
        Dict[str, float]: A dictionary mapping metric names to their values.
    """
    output = subprocess.check_output([
        TREC_EVAL_BIN_PATH,
        qrels_path,
        results_path,
        *shlex.split(flags)
    ]).decode('utf-8')
    return {
        line.split()[0]: (line.split()[2])
        for line in output.splitlines()
    }
# https://github.com/castorini/anserini/blob/master/docs/regressions-beir-v1.0.0-trec-covid-flat.md
# https://github.com/castorini/anserini/blob/master/src/main/resources/regression/beir-v1.0.0-trec-covid-flat.yaml



In [10]:
# -index /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade \
# -topics anserini/src/main/resources/topics-and-qrels/topics.beir-v1.0.0-trec-covid.test.tsv.gz \
!mkdir pv runs
!anserini/target/appassembler/bin/SearchCollection \
-index {PREBUILT_INDEX_DIR} \
-topics trec-covid-queries_trec_format.txt \
-topicreader TsvString \
-output runs/run.beir-v1.0.0-trec-covid-flat.bm25.topics.beir-v1.0.0-trec-covid.test.txt \
-bm25 -removeQuery -hits 1000

2023-04-12 23:28:33,535 INFO  [main] search.SearchCollection (SearchCollection.java:950) - ============ Initializing Searcher ============
2023-04-12 23:28:33,538 INFO  [main] search.SearchCollection (SearchCollection.java:951) - Index: /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade
2023-04-12 23:28:33,715 INFO  [main] search.SearchCollection (SearchCollection.java:955) - Fields: []
2023-04-12 23:28:33,716 INFO  [main] search.SearchCollection (SearchCollection.java:695) - Using DefaultEnglishAnalyzer
2023-04-12 23:28:33,716 INFO  [main] search.SearchCollection (SearchCollection.java:696) - Stemmer: porter
2023-04-12 23:28:33,716 INFO  [main] search.SearchCollection (SearchCollection.java:697) - Keep stopwords? false
2023-04-12 23:28:33,717 INFO  [main] search.SearchCollection (SearchCollection.java:698) - Stopwords file: null
2023-04-12 23:28:33,738 INFO  [main] search.SearchCollection (SearchCollection.java:1227)

In [11]:
#  -c: Average over the complete set of queries in the relevance judgements  
#      instead of the queries in the intersection of relevance judgements 
#      and results.  Missing queries will contribute a value of 0 to all 
#      evaluation measures (which may or may not be reasonable for a  
#      particular evaluation measure, but is reasonable for standard TREC
#      measures.) Default is off.
get_trec_eval_metrics('-c -m ndcg_cut.10', 'trec-covid-qrels_trec_format.txt', 'runs/run.beir-v1.0.0-trec-covid-flat.bm25.topics.beir-v1.0.0-trec-covid.test.txt')

{'ndcg_cut_10': '0.5947'}

The value for ndcg_cut_10 obtained match the expected one.

# Results using corpus expanded with generated queries

## Downloading files locally

In [12]:
!mkdir -pv ids_and_contents generated_queries
!gsutil -m rsync -r gs://marcospiau-doc2query-aula05/ids_and_contents ids_and_contents
!gsutil -m rsync -r gs://marcospiau-doc2query-aula05/generated_queries generated_queries

mkdir: created directory 'ids_and_contents'
mkdir: created directory 'generated_queries'
Building synchronization state...
Starting synchronization...
Copying gs://marcospiau-doc2query-aula05/ids_and_contents/one_line_per_document/content.txt...
Copying gs://marcospiau-doc2query-aula05/ids_and_contents/one_line_per_document/id.txt...
| [2/2 files][185.1 MiB/185.1 MiB] 100% Done                                    
Operation completed over 2 objects/185.1 MiB.                                    
Building synchronization state...
Starting synchronization...
Copying gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-36.txt-1005516...
Copying gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_greedy_decode/predictions.txt-1005516...
Copying gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_beam_search/predictions.txt-1005516...
Copying gs://marcospiau-doc2query-aula05/generated_queries/one_line

In [13]:
!tree -lht ids_and_contents
!tree -lht generated_queries

ids_and_contents
└── [4.0K]  one_line_per_document
    ├── [184M]  content.txt
    └── [1.5M]  id.txt

1 directory, 2 files
generated_queries
├── [4.0K]  one_line_per_document_beam_search
│   ├── [6.8M]  predictions.txt-1005516
│   └── [412K]  one_line_per_document_beam_search.log
├── [4.0K]  one_line_per_document_greedy_decode
│   ├── [6.9M]  predictions.txt-1005516
│   └── [412K]  one_line_per_document_greedy_decode.log
└── [4.0K]  one_line_per_document_topk_sampling
    ├── [6.9M]  predictions-iter-01.txt-1005516
    ├── [7.0M]  predictions-iter-02.txt-1005516
    ├── [6.9M]  predictions-iter-03.txt-1005516
    ├── [7.0M]  predictions-iter-04.txt-1005516
    ├── [7.0M]  predictions-iter-05.txt-1005516
    ├── [7.0M]  predictions-iter-06.txt-1005516
    ├── [7.0M]  predictions-iter-07.txt-1005516
    ├── [7.0M]  predictions-iter-08.txt-1005516
    ├── [7.0M]  predictions-iter-09.txt-1005516
    ├── [7.0M]  predictions-iter-10.txt-1005516
    ├── [7.0M]  predictions-iter-11.txt-100551

There are unk tokens on generated queries:

In [17]:
!grep -ir ⁇ generated_queries | head

generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:what binds with Pol  ⁇ 
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:is oct-4 expressed in LC or  ⁇  lung cancer
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:what is IFN- ⁇  response to adenovirus
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:effects of  ⁇ -secretase activity on gene transcription
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:does rotavirus induce IFN ⁇  production
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:does a  ⁇ -ar polymorphism cause heart rate variability
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:IFN- ⁇  production in NK cells early response to lcmv
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516

Replacing with sed:

In [18]:
!grep -ir ⁇ generated_queries | head | sed s,'⁇',' ',g

generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:what binds with Pol    
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:is oct-4 expressed in LC or     lung cancer
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:what is IFN-    response to adenovirus
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:effects of    -secretase activity on gene transcription
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:does rotavirus induce IFN    production
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:does a    -ar polymorphism cause heart rate variability
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516:IFN-    production in NK cells early response to lcmv
generated_queries/one_line_per_document_topk_sampling/predictions-iter-31.txt-1005516

## Creating dataframe with original corpus and predictions

Data is small enough to be loaded in memory at once, so we will keep it simple and load all data in a single dataframe:

In [19]:
def load_ids(path):
    lines = []
    for line in open(path, 'r'):
        lines.append(line.strip())
    return lines

load_ids('ids_and_contents/one_line_per_document/id.txt')[:10]

['ug7v899j',
 '02tnwd4m',
 'ejv2xln0',
 '2b73a28n',
 '9785vg6d',
 'zjufx4fo',
 '5yhe786e',
 '8zchiykl',
 '8qnrcgnk',
 'jg13scgo']

In [20]:
def read_pred_files(*paths):
    def read_one(path):
        lines = []
        for line in open(path, 'r'):
            line = line.strip()
            line = line.replace('⁇', ' ')
            lines.append(line)
        return lines

    preds = list(map(read_one, paths))

    # assert all files have the same number of lines
    assert len(set(map(len, preds))) == 1
    # merge lines
    preds = list(zip(*preds))
    return preds

read_pred_files(*epath.Path('generated_queries/one_line_per_document_topk_sampling').glob('predictions-*'))[:10]

[('characteristics of mycoplasma pneumoniae',
  'how many patients died of m pneumoniae',
  'how many patients died from m pneumoniae',
  'how many patients die from pneumonia',
  'characteristics of M pneumoniae',
  'how many patients died from pneumonia',
  'frequency of pneumonia in pre-school children',
  'characteristics of all mycoplasma pneumoniae cases',
  'characteristics of mycoplasma pneumoniae',
  'how many people died from pneumonia in the saudi desert',
  'how many patients died from m pneumoniae',
  'study of pneumonia in infants at king abd uah',
  'how many patients died due to pneumonia',
  'how many patients died from pneumonia',
  'how many patients died from m pneumoniae',
  'how many patients died from pneumonia',
  'characteristics of mycoplasma pneumoniae in children',
  'how many patients died from pneumonia',
  'how many patients died from m pneumonia',
  'characteristics of mycoplasma pneumoniae',
  'how many patients died from pneumonia',
  'characteristics 

In [21]:
def load_ids_and_preds(ids_path, *preds_paths):
    ids = load_ids(ids_path)
    preds = read_pred_files(*preds_paths)
    assert len(ids) == len(preds)
    # ids can be duplicated, so we will groupby id and concat all preds for it
    # ids are sorted by construction
    out = {}
    for key, group in itertools.groupby(iterable=zip(ids, preds),
                                        key=lambda x: x[0]):
        out[key] = list(more_itertools.flatten(toolz.pluck(1, group)))
    return out

list(itertools.islice(load_ids_and_preds('ids_and_contents/one_line_per_document/id.txt', *epath.Path('generated_queries/one_line_per_document_topk_sampling').glob('predictions-*')).items(), 10))

[('ug7v899j',
  ['characteristics of mycoplasma pneumoniae',
   'how many patients died of m pneumoniae',
   'how many patients died from m pneumoniae',
   'how many patients die from pneumonia',
   'characteristics of M pneumoniae',
   'how many patients died from pneumonia',
   'frequency of pneumonia in pre-school children',
   'characteristics of all mycoplasma pneumoniae cases',
   'characteristics of mycoplasma pneumoniae',
   'how many people died from pneumonia in the saudi desert',
   'how many patients died from m pneumoniae',
   'study of pneumonia in infants at king abd uah',
   'how many patients died due to pneumonia',
   'how many patients died from pneumonia',
   'how many patients died from m pneumoniae',
   'how many patients died from pneumonia',
   'characteristics of mycoplasma pneumoniae in children',
   'how many patients died from pneumonia',
   'how many patients died from m pneumonia',
   'characteristics of mycoplasma pneumoniae',
   'how many patients died f

## Loading data

In [22]:
# single dataframe 
df_corpus_preds = pl.from_arrow(datasets.load_dataset(
    'BeIR/trec-covid','corpus')['corpus'].data.table)

# concat title and text - this is the baseline text for retrieval
# PS.: during query generation, we used the format
# pl.format('{}. {}', 'title', 'text').alias('title_and_text') to feed T5
# Here, we will simple concatenate the fields using a space

df_corpus_preds = df_corpus_preds.with_columns(
    pl.concat_str('title', 'text', separator=' ').alias('title_and_text'))
df_corpus_preds.head()

Generating corpus split: 0 examples [00:00, ? examples/s]

Dataset trec-covid downloaded and prepared to /root/.cache/huggingface/datasets/BeIR___trec-covid/corpus/0.0.0/093f1fe2ffa7a9c72fa48239c8f279b51d6b171abd77737c7fd1406125307599. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

_id,title,text,title_and_text
str,str,str,str
"""ug7v899j""","""Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia""","""OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) were …","""Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (3…"
"""02tnwd4m""","""Nitric oxide: a pro-inflammatory mediator in lung disease?""","""Inflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO•) and increased indices of NO• -dependent oxidative stress. Although NO• is known to have anti-microbial, anti-inflammatory and anti-oxidant properties, various lines of evidence support the contribution of NO• to lung injury in several disease models. On the basis of biochemical evidence, it is often presumed that such NO• -dependent oxidations are due to the formation of the oxidant peroxynitrite, although alternative mechanisms involving the phagocyte-derived heme proteins myeloperoxidase and eosinophil peroxidase might be operative during conditions of inflammation. Because of the overwhelming literature on NO• generation and activities in the respiratory tract, it would be beyond the scope of this commentary to review this area comprehensively. Instead, it focuses on recent evidence and concepts of the presumed contribution of NO• to inflammatory diseases of the lun…","""Nitric oxide: a pro-inflammatory mediator in lung disease? Inflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO•) and increased indices of NO• -dependent oxidative stress. Although NO• is known to have anti-microbial, anti-inflammatory and anti-oxidant properties, various lines of evidence support the contribution of NO• to lung injury in several disease models. On the basis of biochemical evidence, it is often presumed that such NO• -dependent oxidations are due to the formation of the oxidant peroxynitrite, although alternative mechanisms involving the phagocyte-derived heme proteins myeloperoxidase and eosinophil peroxidase might be

Adding generated queries:

In [23]:
preds_paths_map = {
    x.name: ('ids_and_contents/one_line_per_document/id.txt',
             sorted(x.glob('predictions*')))
    for x in epath.Path('generated_queries').iterdir()
}
preds_paths_map

{'one_line_per_document_topk_sampling': ('ids_and_contents/one_line_per_document/id.txt',
  [PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-01.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-02.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-03.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-04.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-05.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-06.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-07.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictions-iter-08.txt-1005516'),
   PosixGPath('generated_queries/one_line_per_document_topk_sampling/predictio

In [24]:
df_corpus_preds = df_corpus_preds.with_columns(
    pl.col('_id').map_dict(load_ids_and_preds(v[0], *v[1])).alias(f'preds_{k}')
    for k, v in tqdm(preds_paths_map.items())
)


100%|██████████| 3/3 [00:08<00:00,  2.99s/it]


In [26]:
!mkdir -pv backup_dir
df_corpus_preds.write_ipc('backup_dir/df_corpus_preds.arrow')
!ls -lht backup_dir

mkdir: created directory 'backup_dir'
total 879M
-rw-r--r-- 1 root root 879M Apr 12 23:32 df_corpus_preds.arrow


## Creating jsonl files for indexing

In [27]:
def prepare_data_for_indexing(df, col_with_preds=None, max_preds=None, col_with_text='title_and_text'):
    to_concat = []
    if col_with_text is not None:
        to_concat.append(col_with_text)
    if col_with_preds is not None:
        preds_expr = pl.col(col_with_preds)
        if max_preds is not None:
            preds_expr = preds_expr.arr.slice(0, max_preds)
        to_concat.append(preds_expr.arr.join(' '))

    df = df.select(
        pl.col('_id').alias('id'),
        pl.concat_str(to_concat, separator= ' ').alias('contents'))
    return df

prepare_data_for_indexing(df_corpus_preds, 'preds_one_line_per_document_topk_sampling').head(1)

id,contents
str,str
"""ug7v899j""","""Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (3…"


In [28]:
# index with expanded queries_only
prepare_data_for_indexing_expansion_only = functools.partial(
    prepare_data_for_indexing,
    col_with_text=None
)

prepare_data_for_indexing_no_expansion = functools.partial(
    prepare_data_for_indexing,
    col_with_preds=None,
    max_preds=None
)

In [30]:
index_build_fns = {
    'title_and_text_no_expansion': prepare_data_for_indexing_no_expansion}

for pred_col in ['preds_one_line_per_document_greedy_decode', 'preds_one_line_per_document_beam_search', 'preds_one_line_per_document_topk_sampling']:
    if pred_col == 'preds_one_line_per_document_topk_sampling':
        max_preds_range = range(1, 61)
    else:
        max_preds_range = range(1, 2)

    for max_preds in max_preds_range:
        # text and expansion
        index_build_fns[f'{pred_col}_{max_preds}_pred'] = functools.partial(
            prepare_data_for_indexing,
            col_with_preds=pred_col,
            max_preds=max_preds,
            col_with_text='title_and_text')
        # only expansion
        index_build_fns[f'only_preds_{pred_col}_{max_preds}_pred'] = functools.partial(
            prepare_data_for_indexing_expansion_only,
            col_with_preds=pred_col,
            max_preds=max_preds
        )

    del pred_col

len(index_build_fns)

125

In [33]:
index_build_fns['only_preds_preds_one_line_per_document_topk_sampling_18_pred'](df_corpus_preds).head()

id,contents
str,str
"""ug7v899j""","""when did the pneumonia come how many children die from pneumonia characteristics of M pneumoniae characteristics of mycoplasma pneumoniae in children how many patients died from pneumonia characteristics of mycoplasma pneumoniae series of prospective clinical trials on m pneumoniae how many children died from pneumonia how many patients died from m pneumoniae characteristics of mycoplasma pneumoniae what is the incidence of pneumonia in children how many patients died from m pneumoniae how many patients died from pneumonia how many patients died from pneumonia how many patients died from pneumonia how many patients died from pneumonia how many patients died of pneumonia frequency of pneumonia in pre-school children"""
"""02tnwd4m""","""what is NO• in the lung is nitric oxide an inflammatory mediator is nitric oxide a pro-inflammatory mediator what is nitric oxide used for in the lung what is nitric oxide used for in pulmonary disease what is NO• in the lung what is a pro-inflammatory mediator in lung disease what is NO• in the lung inflammatory pathways in lung disease is nitric oxide harmful to the lung what is NO• involved in in the lung what is NO• in the lung is nitric oxide an inflammatory mediator does nitric oxide cause lung inflammation what is NO• in the lung what is NO• an anti-inflammatory agent which of the following is an inflammatory mediator associated with lung injury? what is NO• causing lung inflammation"""
"""ejv2xln0""","""what is surfactant protein d what is surfactant protein d what is surfactant protein d what is surfactant protein d what is surfactant protein d what does surfactant protein d do what is surfactant protein d what is surfactant protein d what is the function of surfactant protein d what is surfactant protein d what molecules are expressed on the surface of surfactant proteins what does surfactant protein d do what is the function of surfactant protein d what is surfactant protein d what does surfactant protein d do what is surfactant protein d what is the function of surfactant protein d what is the function of surfactant protein d"""
"""2b73a28n""","""what is endothelin-1 used for in lung disease what is endothelin-1 what is endothelin-1 what is endothelin-1 implicated in what is endothelin-1 what is endothelin-1 what is endothelin-1 what is endothelin-1 what is endothelin-1 what is endothelin-1 what is endothelin-1 what is the function of endothelin-1 what is endothelin-1 what is endothelin-1 what is endothelin-1 what is the role of endothelin-1 in lung disease what is endothelin-1 used for in lung disease what is endothelin-1 used for in lung disease"""
"""9785vg6d""","""what are the effects of pneumovirus on the respiratory epithelial cells? what does respiratory syncytial virus do what are the effects of pneumovirus infection on the respiratory epithelial cells? what does the respiratory epithelial cells express after pneumovirus infection? what are the effects of pneumovirus infection on the epithelial cells? what are the effects of pneumovirus infection on the respiratory epithelial cells? what is the response of respiratory epithelial cells to pneumovirus infection? what are the effects of pneumovirus on the respiratory epithelial cells? what is the response of respiratory epithelial cells to pneumovirus infection? what are the genotypes of respiratory epithelial cells what does respiratory epithelial cells express after pneumovirus infection what are the effects of pneumovirus infection on the respiratory epithelial cells? what are the effects of pneumovirus on the respiratory epithelial cells? what are the effects of pneumovirus on the respirat…"


In [34]:
!mkdir -pv jsonl_for_indexing

jsonl_for_indexing = {}
for key, fn in tqdm(index_build_fns.items(),
                    desc='Writing jsonl files for indexing'):
    jsonl_path = Path('jsonl_for_indexing').joinpath(key, 'corpus.jsonl')
    jsonl_path.parent.mkdir(parents=True, exist_ok=True)

    fn(df_corpus_preds).write_ndjson(jsonl_path)
    jsonl_for_indexing[key] = str(jsonl_path)
!tree -lht jsonl_for_indexing

mkdir: created directory 'jsonl_for_indexing'


Writing jsonl files for indexing: 100%|██████████| 125/125 [05:05<00:00,  2.44s/it]


jsonl_for_indexing
├── [4.0K]  preds_one_line_per_document_greedy_decode_1_pred
│   └── [195M]  corpus.jsonl
├── [4.0K]  title_and_text_no_expansion
│   └── [188M]  corpus.jsonl
├── [4.0K]  only_preds_preds_one_line_per_document_greedy_decode_1_pred
│   └── [ 12M]  corpus.jsonl
├── [4.0K]  preds_one_line_per_document_beam_search_1_pred
│   └── [195M]  corpus.jsonl
├── [4.0K]  only_preds_preds_one_line_per_document_beam_search_1_pred
│   └── [ 12M]  corpus.jsonl
├── [4.0K]  preds_one_line_per_document_topk_sampling_1_pred
│   └── [195M]  corpus.jsonl
├── [4.0K]  only_preds_preds_one_line_per_document_topk_sampling_1_pred
│   └── [ 12M]  corpus.jsonl
├── [4.0K]  preds_one_line_per_document_topk_sampling_2_pred
│   └── [202M]  corpus.jsonl
├── [4.0K]  only_preds_preds_one_line_per_document_topk_sampling_2_pred
│   └── [ 19M]  corpus.jsonl
├── [4.0K]  preds_one_line_per_document_topk_sampling_3_pred
│   └── [209M]  corpus.jsonl
├── [4.0K]  only_preds_preds_one_line_per_document_topk_sampli

## Creating indexes with anserini

In [38]:
import shlex
import subprocess

ANSERINI_INDEX_COLLECTION_BIN_PATH = 'anserini/target/appassembler/bin/IndexCollection'
DEFAULT_INDEXING_FLAGS = (
    '-collection JsonCollection -generator DefaultLuceneDocumentGenerator '
    '-threads 1 -storePositions -storeDocvectors -storeRaw -optimize')
def create_anserini_index(input_dir, output_dir, flags=DEFAULT_INDEXING_FLAGS):
    """Index jsonl files using anserini.

    Args:
        input_dir: input dir with json corpus
        output_dir: output_dir for index
        flags (str): Flags to pass to IndexCollection binary
    
    Returns:
        command stdout
    """
    output = subprocess.check_output([
        ANSERINI_INDEX_COLLECTION_BIN_PATH,
        '-input', input_dir,
        '-index', output_dir,
        *shlex.split(flags)
    ]).decode('utf-8')
    return output

In [39]:
# args will be input_dir and output_dir
index_args = list(map(lambda x: (x, Path('indexes', x.name)),
                      Path('jsonl_for_indexing').iterdir()))
index_args

[(PosixPath('jsonl_for_indexing/preds_one_line_per_document_topk_sampling_40_pred'),
  PosixPath('indexes/preds_one_line_per_document_topk_sampling_40_pred')),
 (PosixPath('jsonl_for_indexing/only_preds_preds_one_line_per_document_topk_sampling_8_pred'),
  PosixPath('indexes/only_preds_preds_one_line_per_document_topk_sampling_8_pred')),
 (PosixPath('jsonl_for_indexing/only_preds_preds_one_line_per_document_topk_sampling_4_pred'),
  PosixPath('indexes/only_preds_preds_one_line_per_document_topk_sampling_4_pred')),
 (PosixPath('jsonl_for_indexing/only_preds_preds_one_line_per_document_topk_sampling_39_pred'),
  PosixPath('indexes/only_preds_preds_one_line_per_document_topk_sampling_39_pred')),
 (PosixPath('jsonl_for_indexing/only_preds_preds_one_line_per_document_topk_sampling_54_pred'),
  PosixPath('indexes/only_preds_preds_one_line_per_document_topk_sampling_54_pred')),
 (PosixPath('jsonl_for_indexing/preds_one_line_per_document_topk_sampling_36_pred'),
  PosixPath('indexes/preds_one_

In [37]:
%%time
!mkdir -pv indexes
N_WORKERS = 16 ## setting this too high will lead to OOM crash
with mp.Pool(N_WORKERS) as pool:
    print(f'Indexing using a process pool of {N_WORKERS} workers')
    stdouts_indexing = pool.starmap(create_anserini_index, index_args)

mkdir: created directory 'indexes'
Indexing using a process pool of 16 workers
CPU times: user 3.54 s, sys: 2.88 s, total: 6.42 s
Wall time: 8min 56s


In [40]:
df_index_stdouts = pl.DataFrame([
    pl.Series(list(map(lambda x: x[0].name, index_args))).alias('index'),
    pl.Series(stdouts_indexing).alias('stdout')]
)
df_index_stdouts.head()

index,stdout
str,str
"""preds_one_line_per_document_topk_sampling_40_pred""","""2023-04-12 23:38:51,950 INFO [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO 2023-04-12 23:38:51,954 INFO [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer... 2023-04-12 23:38:51,954 INFO [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============ 2023-04-12 23:38:51,955 INFO [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: jsonl_for_indexing/preds_one_line_per_document_topk_sampling_40_pred 2023-04-12 23:38:51,956 INFO [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection 2023-04-12 23:38:51,957 INFO [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator 2023-04-12 23:38:51,958 INFO [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1 2023-04-12 23:38:51,959 INFO [main] index.IndexCollection (IndexCollection.java:389) - Language: en 2023-04-12 2…"
"""only_preds_preds_one_line_per_document_topk_sampling_8_pred""","""2023-04-12 23:40:23,097 INFO [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO 2023-04-12 23:40:23,101 INFO [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer... 2023-04-12 23:40:23,101 INFO [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============ 2023-04-12 23:40:23,102 INFO [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: jsonl_for_indexing/only_preds_preds_one_line_per_document_topk_sampling_8_pred 2023-04-12 23:40:23,103 INFO [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection 2023-04-12 23:40:23,103 INFO [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator 2023-04-12 23:40:23,104 INFO [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1 2023-04-12 23:40:23,104 INFO [main] index.IndexCollection (IndexCollection.java:389) - Language: en 20…"
"""only_preds_preds_one_line_per_document_topk_sampling_4_pred""","""2023-04-12 23:38:51,917 INFO [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO 2023-04-12 23:38:51,921 INFO [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer... 2023-04-12 23:38:51,921 INFO [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============ 2023-04-12 23:38:51,922 INFO [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: jsonl_for_indexing/only_preds_preds_one_line_per_document_topk_sampling_4_pred 2023-04-12 23:38:51,923 INFO [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection 2023-04-12 23:38:51,924 INFO [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator 2023-04-12 23:38:51,925 INFO [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1 2023-04-12 23:38:51,925 INFO [main] index.IndexCollection (IndexCollection.java:389) - Language: en 20…"
"""only_preds_preds_one_line_per_document_topk_sampling_39_pred""","""2023-04-12 23:39:11,210 INFO [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO 2023-04-12 23:39:11,214 INFO [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer... 2023-04-12 23:39:11,214 INFO [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============ 2023-04-12 23:39:11,215 INFO [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: jsonl_for_indexing/only_preds_preds_one_line_per_document_topk_sampling_39_pred 2023-04-12 23:39:11,216 INFO [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection 2023-04-12 23:39:11,217 INFO [main] index.IndexCollection (IndexCol

In [41]:
df_index_stdouts.write_ndjson('backup_dir/index_building_stdouts.jsonl')
!head backup_dir/index_building_stdouts.jsonl

{"index":"preds_one_line_per_document_topk_sampling_40_pred","stdout":"2023-04-12 23:38:51,950 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO\n2023-04-12 23:38:51,954 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...\n2023-04-12 23:38:51,954 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============\n2023-04-12 23:38:51,955 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: jsonl_for_indexing/preds_one_line_per_document_topk_sampling_40_pred\n2023-04-12 23:38:51,956 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection\n2023-04-12 23:38:51,957 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator\n2023-04-12 23:38:51,958 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1\n2023-04-12 23:38:51,959 INFO  [main]

## Generating runs (retrieval)

In [42]:
ANSERINI_SEARCH_COLLECTION_BIN_PATH = 'anserini/target/appassembler/bin/SearchCollection'
DEFAULT_SEARCH_FLAGS = '-topicreader TsvString -bm25 -removeQuery -hits 1000'
def create_run(index_dir, topics_path, results_path,
               flags=DEFAULT_SEARCH_FLAGS):
    """Generate results runs using anserini.

    Args:
        index_dir (str): directory with index
        topics_path (str): path to topics (queries)
        output_file (str): output file with runs (results)
        flags (str): Flags to pass to SearchCollection binary.
    
    Returns:
        command stdout
    """
    output = subprocess.check_output([
        ANSERINI_SEARCH_COLLECTION_BIN_PATH,
        '-index', index_dir,
        '-topics', topics_path,
        '-output', results_path,
        *shlex.split(flags)
    ]).decode('utf-8')
    return output

In [43]:
%%time
!mkdir -pv runs/

create_runs_stdouts = []
for index_dir in tqdm(list(Path('indexes').iterdir())):
    create_runs_stdouts.append({
        'index_dir': index_dir.name,
        'search_results_stdout': create_run(
            index_dir=index_dir,
            topics_path='trec-covid-queries_trec_format.txt',
            results_path=Path('runs') / f'run.{index_dir.name}.txt'
        )
    })

100%|██████████| 125/125 [07:43<00:00,  3.71s/it]

CPU times: user 3.06 s, sys: 17.3 s, total: 20.3 s
Wall time: 7min 43s


In [45]:
df_create_runs_stdout = pl.DataFrame(create_runs_stdouts)
df_create_runs_stdout = df_create_runs_stdout.with_columns(
    pl.col('search_results_stdout').str.extract('(.*?)~(.*?) q/s', 2)
    .apply(float).alias('retrieval_queries_per_second'))
df_create_runs_stdout

index_dir,search_results_stdout,retrieval_queries_per_second
str,str,f64
"""preds_one_line_per_document_topk_sampling_40_pred""","""2023-04-12 23:52:01,348 INFO [main] search.SearchCollection (SearchCollection.java:950) - ============ Initializing Searcher ============ 2023-04-12 23:52:01,351 INFO [main] search.SearchCollection (SearchCollection.java:951) - Index: indexes/preds_one_line_per_document_topk_sampling_40_pred 2023-04-12 23:52:01,733 INFO [main] search.SearchCollection (SearchCollection.java:955) - Fields: [] 2023-04-12 23:52:01,734 INFO [main] search.SearchCollection (SearchCollection.java:695) - Using DefaultEnglishAnalyzer 2023-04-12 23:52:01,734 INFO [main] search.SearchCollection (SearchCollection.java:696) - Stemmer: porter 2023-04-12 23:52:01,735 INFO [main] search.SearchCollection (SearchCollection.java:697) - Keep stopwords? false 2023-04-12 23:52:01,735 INFO [main] search.SearchCollection (SearchCollection.java:698) - Stopwords file: null 2023-04-12 23:52:01,778 INFO [main] search.SearchCollection (SearchCollection.java:1227) - runtag: Anserini 2023-04-12 23:52:01,808 INFO [main] sear…",14.94
"""only_preds_preds_one_line_per_document_topk_sampling_8_pred""","""2023-04-12 23:52:06,059 INFO [main] search.SearchCollection (SearchCollection.java:950) - ============ Initializing Searcher ============ 2023-04-12 23:52:06,062 INFO [main] search.SearchCollection (SearchCollection.java:951) - Index: indexes/only_preds_preds_one_line_per_document_topk_sampling_8_pred 2023-04-12 23:52:06,305 INFO [main] search.SearchCollection (SearchCollection.java:955) - Fields: [] 2023-04-12 23:52:06,306 INFO [main] search.SearchCollection (SearchCollection.java:695) - Using DefaultEnglishAnalyzer 2023-04-12 23:52:06,307 INFO [main] search.SearchCollection (SearchCollection.java:696) - Stemmer: porter 2023-04-12 23:52:06,307 INFO [main] search.SearchCollection (SearchCollection.java:697) - Keep stopwords? false 2023-04-12 23:52:06,307 INFO [main] search.SearchCollection (SearchCollection.java:698) - Stopwords file: null 2023-04-12 23:52:06,330 INFO [main] search.SearchCollection (SearchCollection.java:1227) - runtag: Anserini 2023-04-12 23:52:06,332 INFO […",26.23
"""only_preds_preds_one_line_per_document_topk_sampling_4_pred""","""2023-04-12 23:52:09,179 INFO [main] search.SearchCollection (SearchCollection.java:950) - ============ Initializing Searcher ============ 2023-04-12 23:52:09,182 INFO [main] search.SearchCollection (SearchCollection.java:951) - Index: indexes/only_preds_preds_one_line_per_document_topk_sampling_4_pred 2023-04-12 23:52:09,417 INFO [main] search.SearchCollection (SearchCollection.java:955) - Fields: [] 2023-04-12 23:52:09,419 INFO [main] search.SearchCollection (SearchCollection.java:695) - Using DefaultEnglishAnalyzer 2023-04-12 23:52:09,419 INFO [main] search.SearchCollection (SearchCollection.java:696) - Stemmer: porter 2023-04-12 23:52:09,419 INFO [main] search.SearchCollection (SearchCollection.java:697) - Keep stopwords? false 2023-04-12 23:52:09,420 INFO [main] search.SearchCollection (SearchCollection.java:698) - Stopwords file: null 2023-04-12 23:52:09,443 INFO [main] search.SearchCollection (SearchCollection.java:1227) - runtag: Anserini 2023-04-12 23:52:09,445 INFO […",33.74
"""only_preds_preds_one_line_per_document_topk_sampling_39_pred""","""2023-04-12 23:52:11,821 INFO [main] search.SearchCollection (SearchCollection.java:950) - ============ Initializing Searcher ============ 2023-04-12 23:52:11,824 INFO [main] search.SearchCollection (SearchCollection.java:951) - Index: indexes/only_preds_preds_one_line_per_document_topk_sampling_39_pred 2023-04-12 23:52:12,068 INFO [main] search.SearchCollection (SearchCollection.java:955) - Fields: [] 2023-04-12 23:52:12,069 INFO [main] search.SearchCollection (SearchCollection.java:695) - Using DefaultEnglishAnalyzer 2023-04-12 23:52:12,069 INFO [main] search.SearchCollection (SearchCollection.java:696) - Stemmer: porter 2023-04-12 23:52:12,069 INFO [mai

In [46]:
df_create_runs_stdout.write_ndjson('backup_dir/df_create_runs_stdout.jsonl')
!head backup_dir/df_create_runs_stdout.jsonl
!ls -lht backup_dir

{"index_dir":"preds_one_line_per_document_topk_sampling_40_pred","search_results_stdout":"2023-04-12 23:52:01,348 INFO  [main] search.SearchCollection (SearchCollection.java:950) - ============ Initializing Searcher ============\n2023-04-12 23:52:01,351 INFO  [main] search.SearchCollection (SearchCollection.java:951) - Index: indexes/preds_one_line_per_document_topk_sampling_40_pred\n2023-04-12 23:52:01,733 INFO  [main] search.SearchCollection (SearchCollection.java:955) - Fields: []\n2023-04-12 23:52:01,734 INFO  [main] search.SearchCollection (SearchCollection.java:695) - Using DefaultEnglishAnalyzer\n2023-04-12 23:52:01,734 INFO  [main] search.SearchCollection (SearchCollection.java:696) - Stemmer: porter\n2023-04-12 23:52:01,735 INFO  [main] search.SearchCollection (SearchCollection.java:697) - Keep stopwords? false\n2023-04-12 23:52:01,735 INFO  [main] search.SearchCollection (SearchCollection.java:698) - Stopwords file: null\n2023-04-12 23:52:01,778 INFO  [main] search.SearchColl

## Calculating metrics for runs

In [47]:
eval_metrics_fn = functools.partial(
    get_trec_eval_metrics,
    '-c -m ndcg_cut.10',
    'trec-covid-qrels_trec_format.txt'    
)

In [48]:
%%time
df_results = pl.Series(map(str, Path('runs').iterdir())).to_frame('run_path')
df_results = df_results.with_columns(pl.col('run_path').apply(eval_metrics_fn).alias('metrics'))
df_results = (df_results
              .unnest('metrics')
              .with_columns(pl.col('ndcg_cut_10').apply(float)))
df_results = df_results.with_columns(pl.col('run_path').str.extract('/run.(.*?).txt').alias('run_name'))
df_results = df_results.join(
    df_create_runs_stdout.select(pl.col('index_dir').alias('run_name'),
                                 'retrieval_queries_per_second'),
                             on='run_name', how='left')
df_results

CPU times: user 302 ms, sys: 11.2 s, total: 11.5 s
Wall time: 25 s


run_path,ndcg_cut_10,run_name,retrieval_queries_per_second
str,f64,str,f64
"""runs/run.only_preds_preds_one_line_per_document_topk_sampling_11_pred.txt""",0.4992,"""only_preds_preds_one_line_per_document_topk_sampling_11_pred""",31.95
"""runs/run.title_and_text_no_expansion.txt""",0.5947,"""title_and_text_no_expansion""",21.97
"""runs/run.only_preds_preds_one_line_per_document_topk_sampling_57_pred.txt""",0.5598,"""only_preds_preds_one_line_per_document_topk_sampling_57_pred""",21.9
"""runs/run.preds_one_line_per_document_topk_sampling_59_pred.txt""",0.6353,"""preds_one_line_per_document_topk_sampling_59_pred""",14.4
"""runs/run.preds_one_line_per_document_topk_sampling_55_pred.txt""",0.6304,"""preds_one_line_per_document_topk_sampling_55_pred""",13.5
"""runs/run.preds_one_line_per_document_topk_sampling_57_pred.txt""",0.6388,"""preds_one_line_per_document_topk_sampling_57_pred""",13.92
"""runs/run.only_preds_preds_one_line_per_document_topk_sampling_5_pred.txt""",0.4565,"""only_preds_preds_one_line_per_document_topk_sampling_5_pred""",35.49
"""runs/run.preds_one_line_per_document_topk_sampling_47_pred.txt""",0.6393,"""preds_one_line_per_document_topk_sampling_47_pred""",14.64
"""runs/run.preds_one_line_per_document_topk_sampling_9_pred.txt""",0.6594,"""preds_one_line_per_document_topk_sampling_9_pred""",18.48


In [76]:
df_results.sort('ndcg_cut_10').to_pandas()

,run_path,ndcg_cut_10,run_name,retrieval_queries_per_second
0,runs/run.only_preds_preds_one_line_per_document_beam_search_1_pred.txt,0.3384,only_preds_preds_one_line_per_document_beam_search_1_pred,34.67
1,runs/run.only_preds_preds_one_line_per_document_greedy_decode_1_pred.txt,0.3523,only_preds_preds_one_line_per_document_greedy_decode_1_pred,31.61
2,runs/run.only_preds_preds_one_line_per_document_topk_sampling_1_pred.txt,0.3557,only_preds_preds_one_line_per_document_topk_sampling_1_pred,32.40
3,runs/run.only_preds_preds_one_line_per_document_topk_sampling_2_pred.txt,0.4191,only_preds_preds_one_line_per_document_topk_sampling_2_pred,27.44
4,runs/run.only_preds_preds_one_line_per_document_topk_sampling_3_pred.txt,0.4398,only_preds_preds_one_line_per_document_topk_sampling_3_pred,30.62
5,runs/run.only_preds_preds_one_line_per_document_topk_sampling_4_pred.txt,0.4507,only_preds_preds_one_line_per_document_topk_sampling_4_pred,33.74
6,runs/run.only_preds_preds_one_line_per_document_topk_sampling_5_pred.txt,0.4565,only_preds_preds_one_line_per_document_topk_sampling_5_pred,35.49
7,runs/run.only_preds_preds_one_line_per_document_topk_sampling_6_pred.txt,0.4800,only_preds_preds_one_line_per_document_topk_sampling_6_pred,27.26
8,runs/run.only_preds_preds_one_line_per_document_topk_sampling_7_pred.txt,0.4865,only_preds_preds_one_line_per_document_topk_sampling_7_pred,24.62
9,runs/run.only_preds_preds_one_line_per_document_topk_sampling_10_pred.txt,0.4976,only_preds_preds_one_line_per_document_topk_sampling_10_pred,24.68


In [100]:
import re

In [101]:
re.match('^(.*?)_'only_preds_preds_one_line_per_document_topk_sampling_57_pred')

<module 're' from '/usr/lib/python3.9/re.py'>

In [142]:
def format_results_df(df):
    decoding_expr = (
        pl.when(pl.col('run_name').str.contains('greedy')).then('greedy')
        .when(pl.col('run_name').str.contains('beam'))
        .then('beam').when(pl.col('run_name').str.contains('topk')).then('topk')
        .when(pl.col('run_name').str.contains('no_expansion')).then('no_expansion')
        .otherwise('NADA')).alias('decoding')
    is_only_preds_expr = pl.col('run_name').str.contains(
        'only_preds').alias('is_only_preds')
    n_preds_expr = pl.col('run_name').str.extract(
        '^(.*?)_(\d+)_pred$', 2).cast(pl.Int32).alias('n_preds')

    df = df.select(
        'run_name',
        decoding_expr, is_only_preds_expr, n_preds_expr,
        'ndcg_cut_10',
        pl.col('retrieval_queries_per_second').alias('queries_per_second'),
        (1 / pl.col('retrieval_queries_per_second')).alias('seconds_per_query'))
    df = df.sort('decoding', 'is_only_preds', 'n_preds')
    return df
df_results_2 = format_results_df(df_results)
df_results_2

run_name,decoding,is_only_preds,n_preds,ndcg_cut_10,queries_per_second,seconds_per_query
str,str,bool,i32,f64,f64,f64
"""beir-v1.0.0-trec-covid-flat.bm25.topics.beir-v1.0.0-trec-covid.test""","""NADA""",false,null,0.5947,null,null
"""preds_one_line_per_document_beam_search_1_pred""","""beam""",false,1,0.6635,20.8,0.048077
"""only_preds_preds_one_line_per_document_beam_search_1_pred""","""beam""",true,1,0.3384,34.67,0.028843
"""preds_one_line_per_document_greedy_decode_1_pred""","""greedy""",false,1,0.6598,17.9,0.055866
"""only_preds_preds_one_line_per_document_greedy_decode_1_pred""","""greedy""",true,1,0.3523,31.61,0.031636
"""title_and_text_no_expansion""","""no_expansion""",false,null,0.5947,21.97,0.045517
"""preds_one_line_per_document_topk_sampling_1_pred""","""topk""",false,1,0.6445,19.92,0.050201
"""preds_one_line_per_document_topk_sampling_2_pred""","""topk""",false,2,0.6427,17.74,0.05637
"""preds_one_line_per_document_topk_sampling_3_pred""","""topk""",false,3,0.6489,18.8,0.053191


In [ ]:
df_results_2.select(pl.col('n_preds').suffix('s'))

In [213]:
def format_to_plot(df):
    suffixes = {True: '_only_preds', False: '_text_and_preds'}
    cols_to_suffix = ['ndcg_cut_10', 'queries_per_second', 'seconds_per_query']
    key_cols = ['n_preds', 'decoding']
    to_join = []
    for key, group in df.groupby('is_only_preds'):
        suffix = suffixes[key]
        group = group.select(pl.col(key_cols), pl.col(cols_to_suffix).suffix(suffix))
        to_join.append(group)
    left, right = to_join
    out = left.join(right, on=key_cols, how='outer')
    out = out.sort('n_preds')
    return out




    # return bys.keys()

df_results_3 = format_to_plot(df_results_2)
df_results_3

n_preds,decoding,ndcg_cut_10_only_preds,queries_per_second_only_preds,seconds_per_query_only_preds,ndcg_cut_10_text_and_preds,queries_per_second_text_and_preds,seconds_per_query_text_and_preds
i32,str,f64,f64,f64,f64,f64,f64
null,"""NADA""",null,null,null,0.5947,null,null
null,"""no_expansion""",null,null,null,0.5947,21.97,0.045517
1,"""beam""",0.3384,34.67,0.028843,0.6635,20.8,0.048077
1,"""greedy""",0.3523,31.61,0.031636,0.6598,17.9,0.055866
1,"""topk""",0.3557,32.4,0.030864,0.6445,19.92,0.050201
2,"""topk""",0.4191,27.44,0.036443,0.6427,17.74,0.05637
3,"""topk""",0.4398,30.62,0.032658,0.6489,18.8,0.053191
4,"""topk""",0.4507,33.74,0.029638,0.6588,19.39,0.051573
5,"""topk""",0.4565,35.49,0.028177,0.6599,18.53,0.053967


In [218]:
df_results_3.columns

['n_preds',
 'decoding',
 'ndcg_cut_10_only_preds',
 'queries_per_second_only_preds',
 'seconds_per_query_only_preds',
 'ndcg_cut_10_text_and_preds',
 'queries_per_second_text_and_preds',
 'seconds_per_query_text_and_preds']

In [221]:
def pivot_plot(df, value_col):
    df_pivot = df.pivot(index='n_preds', columns='decoding', values=value_col,
                        aggregate_function=None)
    return df_pivot

pivot_plot(df_results_3, 'ndcg_cut_10_text_and_preds').write_csv()

'n_preds,NADA,no_expansion,beam,greedy,topk\n,0.5947,0.5947,,,\n1,,,0.6635,0.6598,0.6445\n2,,,,,0.6427\n3,,,,,0.6489\n4,,,,,0.6588\n5,,,,,0.6599\n6,,,,,0.6625\n7,,,,,0.67\n8,,,,,0.6685\n9,,,,,0.6594\n10,,,,,0.6593\n11,,,,,0.6679\n12,,,,,0.6639\n13,,,,,0.664\n14,,,,,0.6656\n15,,,,,0.6674\n16,,,,,0.6675\n17,,,,,0.6704\n18,,,,,0.6782\n19,,,,,0.6741\n20,,,,,0.6748\n21,,,,,0.6774\n22,,,,,0.6693\n23,,,,,0.6696\n24,,,,,0.6708\n25,,,,,0.6667\n26,,,,,0.6767\n27,,,,,0.6745\n28,,,,,0.6719\n29,,,,,0.6681\n30,,,,,0.6673\n31,,,,,0.6641\n32,,,,,0.659\n33,,,,,0.6552\n34,,,,,0.6553\n35,,,,,0.6546\n36,,,,,0.658\n37,,,,,0.651\n38,,,,,0.6495\n39,,,,,0.6462\n40,,,,,0.6426\n41,,,,,0.6442\n42,,,,,0.6407\n43,,,,,0.6376\n44,,,,,0.6399\n45,,,,,0.641\n46,,,,,0.6429\n47,,,,,0.6393\n48,,,,,0.6388\n49,,,,,0.6358\n50,,,,,0.6395\n51,,,,,0.6359\n52,,,,,0.6395\n53,,,,,0.6371\n54,,,,,0.6334\n55,,,,,0.6304\n56,,,,,0.6373\n57,,,,,0.6388\n58,,,,,0.6355\n59,,,,,0.6353\n60,,,,,0.6338\n'

In [227]:
print(pivot_plot(df_results_3, 'ndcg_cut_10_text_and_preds').head())
print(pivot_plot(df_results_3, 'ndcg_cut_10_only_preds').head())

shape: (5, 6)
┌─────────┬────────┬──────────────┬────────┬────────┬────────┐
│ n_preds ┆ NADA   ┆ no_expansion ┆ beam   ┆ greedy ┆ topk   │
│ ---     ┆ ---    ┆ ---          ┆ ---    ┆ ---    ┆ ---    │
│ i32     ┆ f64    ┆ f64          ┆ f64    ┆ f64    ┆ f64    │
╞═════════╪════════╪══════════════╪════════╪════════╪════════╡
│ null    ┆ 0.5947 ┆ 0.5947       ┆ null   ┆ null   ┆ null   │
│ 1       ┆ null   ┆ null         ┆ 0.6635 ┆ 0.6598 ┆ 0.6445 │
│ 2       ┆ null   ┆ null         ┆ null   ┆ null   ┆ 0.6427 │
│ 3       ┆ null   ┆ null         ┆ null   ┆ null   ┆ 0.6489 │
│ 4       ┆ null   ┆ null         ┆ null   ┆ null   ┆ 0.6588 │
└─────────┴────────┴──────────────┴────────┴────────┴────────┘
shape: (5, 6)
┌─────────┬──────┬──────────────┬────────┬────────┬────────┐
│ n_preds ┆ NADA ┆ no_expansion ┆ beam   ┆ greedy ┆ topk   │
│ ---     ┆ ---  ┆ ---          ┆ ---    ┆ ---    ┆ ---    │
│ i32     ┆ f64  ┆ f64          ┆ f64    ┆ f64    ┆ f64    │
╞═════════╪══════╪══════════════╪══

In [222]:
pivot_plot(df_results_3, 'ndcg_cut_10_only_preds').write_csv()

'n_preds,NADA,no_expansion,beam,greedy,topk\n,,,,,\n1,,,0.3384,0.3523,0.3557\n2,,,,,0.4191\n3,,,,,0.4398\n4,,,,,0.4507\n5,,,,,0.4565\n6,,,,,0.48\n7,,,,,0.4865\n8,,,,,0.5038\n9,,,,,0.5007\n10,,,,,0.4976\n11,,,,,0.4992\n12,,,,,0.5001\n13,,,,,0.505\n14,,,,,0.5086\n15,,,,,0.5238\n16,,,,,0.5108\n17,,,,,0.5176\n18,,,,,0.5189\n19,,,,,0.5353\n20,,,,,0.5351\n21,,,,,0.5416\n22,,,,,0.5423\n23,,,,,0.5463\n24,,,,,0.5488\n25,,,,,0.5425\n26,,,,,0.5425\n27,,,,,0.5419\n28,,,,,0.5448\n29,,,,,0.5418\n30,,,,,0.5493\n31,,,,,0.5483\n32,,,,,0.5487\n33,,,,,0.5527\n34,,,,,0.5545\n35,,,,,0.563\n36,,,,,0.5611\n37,,,,,0.5641\n38,,,,,0.5638\n39,,,,,0.5554\n40,,,,,0.5577\n41,,,,,0.5594\n42,,,,,0.5554\n43,,,,,0.5553\n44,,,,,0.5556\n45,,,,,0.5557\n46,,,,,0.5557\n47,,,,,0.5582\n48,,,,,0.5608\n49,,,,,0.5546\n50,,,,,0.557\n51,,,,,0.5545\n52,,,,,0.5593\n53,,,,,0.5577\n54,,,,,0.5552\n55,,,,,0.5604\n56,,,,,0.5605\n57,,,,,0.5598\n58,,,,,0.5596\n59,,,,,0.5602\n60,,,,,0.5566\n'

Preds + expansion:

In [178]:
df_results_2.filter(pl.col('is_only_preds')).pivot(index='n_preds', columns='decoding', values='ndcg_cut_10', aggregate_function=None).write_csv()

'n_preds,beam,greedy,topk\n1,0.3384,0.3523,0.3557\n2,,,0.4191\n3,,,0.4398\n4,,,0.4507\n5,,,0.4565\n6,,,0.48\n7,,,0.4865\n8,,,0.5038\n9,,,0.5007\n10,,,0.4976\n11,,,0.4992\n12,,,0.5001\n13,,,0.505\n14,,,0.5086\n15,,,0.5238\n16,,,0.5108\n17,,,0.5176\n18,,,0.5189\n19,,,0.5353\n20,,,0.5351\n21,,,0.5416\n22,,,0.5423\n23,,,0.5463\n24,,,0.5488\n25,,,0.5425\n26,,,0.5425\n27,,,0.5419\n28,,,0.5448\n29,,,0.5418\n30,,,0.5493\n31,,,0.5483\n32,,,0.5487\n33,,,0.5527\n34,,,0.5545\n35,,,0.563\n36,,,0.5611\n37,,,0.5641\n38,,,0.5638\n39,,,0.5554\n40,,,0.5577\n41,,,0.5594\n42,,,0.5554\n43,,,0.5553\n44,,,0.5556\n45,,,0.5557\n46,,,0.5557\n47,,,0.5582\n48,,,0.5608\n49,,,0.5546\n50,,,0.557\n51,,,0.5545\n52,,,0.5593\n53,,,0.5577\n54,,,0.5552\n55,,,0.5604\n56,,,0.5605\n57,,,0.5598\n58,,,0.5596\n59,,,0.5602\n60,,,0.5566\n'

In [152]:
df_results_2.pivot(index='n_preds', columns='decoding', values='ndcg_cut_10', aggregate_function=pl.count())

n_preds,NADA,beam,greedy,no_expansion,topk
i32,u32,u32,u32,u32,u32
null,1,null,null,1,null
1,null,2,2,null,2
2,null,null,null,null,2
3,null,null,null,null,2
4,null,null,null,null,2
5,null,null,null,null,2
6,null,null,null,null,2
7,null,null,null,null,2
8,null,null,null,null,2


In [146]:
df_results.pivot?

In [143]:
df_results_2.write_csv()

'run_name,decoding,is_only_preds,n_preds,ndcg_cut_10,queries_per_second,seconds_per_query\nbeir-v1.0.0-trec-covid-flat.bm25.topics.beir-v1.0.0-trec-covid.test,NADA,false,,0.5947,,\npreds_one_line_per_document_beam_search_1_pred,beam,false,1,0.6635,20.8,0.04807692307692307\nonly_preds_preds_one_line_per_document_beam_search_1_pred,beam,true,1,0.3384,34.67,0.028843380444188056\npreds_one_line_per_document_greedy_decode_1_pred,greedy,false,1,0.6598,17.9,0.0558659217877095\nonly_preds_preds_one_line_per_document_greedy_decode_1_pred,greedy,true,1,0.3523,31.61,0.03163555836760519\ntitle_and_text_no_expansion,no_expansion,false,,0.5947,21.97,0.04551661356395084\npreds_one_line_per_document_topk_sampling_1_pred,topk,false,1,0.6445,19.92,0.0502008032128514\npreds_one_line_per_document_topk_sampling_2_pred,topk,false,2,0.6427,17.74,0.056369785794813984\npreds_one_line_per_document_topk_sampling_3_pred,topk,false,3,0.6489,18.8,0.05319148936170213\npreds_one_line_per_document_topk_sampling_4_pred

In [52]:
df_results.write_csv()

'run_path,ndcg_cut_10,run_name,retrieval_queries_per_second\nruns/run.only_preds_preds_one_line_per_document_topk_sampling_11_pred.txt,0.4992,only_preds_preds_one_line_per_document_topk_sampling_11_pred,31.95\nruns/run.title_and_text_no_expansion.txt,0.5947,title_and_text_no_expansion,21.97\nruns/run.only_preds_preds_one_line_per_document_topk_sampling_57_pred.txt,0.5598,only_preds_preds_one_line_per_document_topk_sampling_57_pred,21.9\nruns/run.preds_one_line_per_document_topk_sampling_59_pred.txt,0.6353,preds_one_line_per_document_topk_sampling_59_pred,14.4\nruns/run.preds_one_line_per_document_topk_sampling_55_pred.txt,0.6304,preds_one_line_per_document_topk_sampling_55_pred,13.5\nruns/run.preds_one_line_per_document_topk_sampling_57_pred.txt,0.6388,preds_one_line_per_document_topk_sampling_57_pred,13.92\nruns/run.only_preds_preds_one_line_per_document_topk_sampling_5_pred.txt,0.4565,only_preds_preds_one_line_per_document_topk_sampling_5_pred,35.49\nruns/run.preds_one_line_per_docum

In [68]:
df_results.write_ndjson('backup_dir/df_results.jsonl')
!ls -lht backup_dir

total 879M
-rw-r--r-- 1 root root  26K Apr 13 00:05 df_results.jsonl
-rw-r--r-- 1 root root 193K Apr 13 00:01 df_create_runs_stdout.jsonl
-rw-r--r-- 1 root root 565K Apr 12 23:51 index_building_stdouts.jsonl
-rw-r--r-- 1 root root 879M Apr 12 23:32 df_corpus_preds.arrow


In [69]:
!head jsonl_for_indexing/only_preds_preds_one_line_per_document_topk_sampling_60_pred/*

{"id":"ug7v899j","contents":"when did the pneumonia come how many children die from pneumonia characteristics of M pneumoniae characteristics of mycoplasma pneumoniae in children how many patients died from pneumonia characteristics of mycoplasma pneumoniae series of prospective clinical trials on m pneumoniae how many children died from pneumonia how many patients died from m pneumoniae characteristics of mycoplasma pneumoniae what is the incidence of pneumonia in children how many patients died from m pneumoniae how many patients died from pneumonia how many patients died from pneumonia how many patients died from pneumonia how many patients died from pneumonia how many patients died of pneumonia frequency of pneumonia in pre-school children case series of mycoplasma pneumoniae how many patients died from m pneumonia how many patients died from m pneumoniae how many patients died from m pneumoniae how many patients died of m pneumoniae how many people died from pneumonia in the saudi

In [70]:
!cp -v trec-covid-queries_trec_format.txt trec-covid-qrels_trec_format.txt backup_dir
!ls -lht backup_dir

'trec-covid-queries_trec_format.txt' -> 'backup_dir/trec-covid-queries_trec_format.txt'
'trec-covid-qrels_trec_format.txt' -> 'backup_dir/trec-covid-qrels_trec_format.txt'
total 880M
-rw-r--r-- 1 root root 1023K Apr 13 00:05 trec-covid-qrels_trec_format.txt
-rw-r--r-- 1 root root  3.6K Apr 13 00:05 trec-covid-queries_trec_format.txt
-rw-r--r-- 1 root root   26K Apr 13 00:05 df_results.jsonl
-rw-r--r-- 1 root root  193K Apr 13 00:01 df_create_runs_stdout.jsonl
-rw-r--r-- 1 root root  565K Apr 12 23:51 index_building_stdouts.jsonl
-rw-r--r-- 1 root root  879M Apr 12 23:32 df_corpus_preds.arrow


## Backing up data to GCP for later

In [71]:
%%time
!gsutil -m rsync -r indexes gs://marcospiau-doc2query-aula05/indexes
!gsutil -m rsync -r runs gs://marcospiau-doc2query-aula05/runs
!gsutil -m rsync -r backup_dir gs://marcospiau-doc2query-aula05/backup_dir
!gsutil -m rsync -r jsonl_for_indexing gs://marcospiau-doc2query-aula05/jsonl_for_indexing

Building synchronization state...
Starting synchronization...
Copying file://indexes/only_preds_preds_one_line_per_document_beam_search_1_pred/_0.fdx [Content-Type=application/octet-stream]...
Copying file://indexes/only_preds_preds_one_line_per_document_greedy_decode_1_pred/_0.fdx [Content-Type=application/octet-stream]...
Copying file://indexes/only_preds_preds_one_line_per_document_beam_search_1_pred/_0_Lucene90_0.doc [Content-Type=application/msword]...
Copying file://indexes/only_preds_preds_one_line_per_document_beam_search_1_pred/_0_Lucene90_0.tip [Content-Type=application/octet-stream]...
Copying file://indexes/only_preds_preds_one_line_per_document_beam_search_1_pred/_0.tvd [Content-Type=application/octet-stream]...
Copying file://indexes/only_preds_preds_one_line_per_document_beam_search_1_pred/_0.nvd [Content-Type=application/octet-stream]...
Copying file://indexes/only_preds_preds_one_line_per_document_beam_search_1_pred/_0.fdt [Content-Type=application/octet-stream]...
Cop

Total size:

In [74]:
!gsutil du -shc gs://marcospiau-doc2query-aula05

114.48 GiB   gs://marcospiau-doc2query-aula05
114.48 GiB   total


In [ ]:
!gsutil du -shc gs://marcospiau-doc2query-aula05

60.46 GiB    gs://marcospiau-doc2query-aula05
60.46 GiB    total
